
# Advanced Graph Analysis & NLP

In this notebook, we will be using a combination of Natural Language Processing and network analysis to look at a Mafia network. The settings the mafia network is the following: As members of an investigation unit, we have been observing a network of families, which have been associated with nefarious activities. We will use two datasets:

1. PDF files written by an undercover agent
2. A network of interactions between those families

For NLP in Spark we can use the open-source *spark-nlp* library, which allows us to use a variety of Deep Learning models.

Since here we are given several PDFs to work with, we need a Python library to parse them using ``UDF``. We first need to install the external Python library ``pypdf``. The *graphframes* library, which we used in the previous lab, offers the useful ``GraphFrame``, but choices for graph algorithms are relatively limited. Thus, we will install the ``networkx`` library, which offers a range of popular graph algorithms.

In [0]:
!pip3 install pypdf networkx

     |████████████████████████████████| 290 kB 4.6 MB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-482a319a-f728-4057-acd9-8b09e94dff36/bin/python -m pip install --upgrade pip' command.


In [0]:
import io
import numpy as np
import networkx as nx
from pypdf import PdfReader 

We install the ``spark-nlp`` dependencies next. 

In [0]:
import sparknlp

# Start Spark Session
spark = sparknlp.start()

from sparknlp.base import DocumentAssembler, Pipeline, LightPipeline
from sparknlp.annotator import (
    Tokenizer,
    WordEmbeddingsModel,
    NerDLModel,
    NerConverter
)

import pyspark.sql.functions as F

We load all the reports from our agent as PDF files.

In [0]:
mafia_network_communications = spark.read.format("binaryFile").load("dbfs:/FileStore/crime_letters/*.pdf")

In [0]:
mafia_network_communications.display()

path,modificationTime,length,content
dbfs:/FileStore/crime_letters/communication_22_5_2013.pdf,2024-05-06T22:50:58.000+0000,142828,JVBERi0xLjQNCiXi48/TDQoxIDAgb2JqDQo8PA0KL1R5cGUgL1BhZ2UNCi9NZWRpYUJveCBbIDAgMCA1OTUuMzA0IDg0MS44OSBdDQovUmVzb3VyY2VzIDw8IC9YT2JqZWN0IDw8IC9JbTQxIDIgMCBSID4+IC9Gb250IDw8IC8= (truncated)
dbfs:/FileStore/crime_letters/communication_15_4_2013.pdf,2024-05-06T22:50:58.000+0000,91833,JVBERi0xLjQNCiXi48/TDQoxIDAgb2JqDQo8PA0KL1R5cGUgL1BhZ2UNCi9NZWRpYUJveCBbIDAgMCA1OTUuMzA0IDg0MS44OSBdDQovUmVzb3VyY2VzIDw8IC9YT2JqZWN0IDw8IC9JbTIzIDIgMCBSID4+IC9Gb250IDw8IC8= (truncated)
dbfs:/FileStore/crime_letters/communication_2_3_2013.pdf,2024-05-06T22:50:57.000+0000,89489,JVBERi0xLjQNCiXi48/TDQoxIDAgb2JqDQo8PA0KL1R5cGUgL1BhZ2UNCi9NZWRpYUJveCBbIDAgMCA1OTUuMzA0IDg0MS44OSBdDQovUmVzb3VyY2VzIDw8IC9YT2JqZWN0IDw8IC9JbTUgMiAwIFIgPj4gL0ZvbnQgPDwgL0Y= (truncated)
dbfs:/FileStore/crime_letters/communication_19_4_2013.pdf,2024-05-06T22:50:58.000+0000,27676,JVBERi0xLjQNCiXi48/TDQoxIDAgb2JqDQo8PA0KL1R5cGUgL1BhZ2UNCi9NZWRpYUJveCBbIDAgMCA1OTUuMzA0IDg0MS44OSBdDQovUmVzb3VyY2VzIDw8IC9Gb250IDw8IC9GMSAyIDAgUiA+PiA+Pg0KL0NvbnRlbnRzIDM= (truncated)
dbfs:/FileStore/crime_letters/communication_3_4_2013.pdf,2024-05-06T22:50:57.000+0000,27587,JVBERi0xLjQNCiXi48/TDQoxIDAgb2JqDQo8PA0KL1R5cGUgL1BhZ2UNCi9NZWRpYUJveCBbIDAgMCA1OTUuMzA0IDg0MS44OSBdDQovUmVzb3VyY2VzIDw8IC9Gb250IDw8IC9GMSAyIDAgUiA+PiA+Pg0KL0NvbnRlbnRzIDM= (truncated)
dbfs:/FileStore/crime_letters/communication_17_4_2013.pdf,2024-05-06T22:50:58.000+0000,27535,JVBERi0xLjQNCiXi48/TDQoxIDAgb2JqDQo8PA0KL1R5cGUgL1BhZ2UNCi9NZWRpYUJveCBbIDAgMCA1OTUuMzA0IDg0MS44OSBdDQovUmVzb3VyY2VzIDw8IC9Gb250IDw8IC9GMSAyIDAgUiA+PiA+Pg0KL0NvbnRlbnRzIDM= (truncated)
dbfs:/FileStore/crime_letters/communication_6_4_2013.pdf,2024-05-06T22:50:58.000+0000,27517,JVBERi0xLjQNCiXi48/TDQoxIDAgb2JqDQo8PA0KL1R5cGUgL1BhZ2UNCi9NZWRpYUJveCBbIDAgMCA1OTUuMzA0IDg0MS44OSBdDQovUmVzb3VyY2VzIDw8IC9Gb250IDw8IC9GMSAyIDAgUiA+PiA+Pg0KL0NvbnRlbnRzIDM= (truncated)


As the next step, we define a Python UDF that takes the binary content of each PDF and convert it to text.

In [0]:
@udf
def pdf_to_text(pdf) -> str:
    """
    We transform a PDF (binary) into a string. The "content" column is already binary, so we read the bytes directly.
    """

    # First we load the binary content
    bytes_stream = io.BytesIO(pdf)

    # We initialize the reader
    reader = PdfReader(bytes_stream)

    # As the final step, we go over each page (note though that in our case our PDFs have only one page each) and extract the text
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"

    return text

In [0]:
mafia_network_communications = mafia_network_communications \
  .withColumn("report_text", pdf_to_text(F.col("content")))

In [0]:
mafia_network_communications.display()

path,modificationTime,length,content,report_text
dbfs:/FileStore/crime_letters/communication_22_5_2013.pdf,2024-05-06T22:50:58.000+0000,142828,JVBERi0xLjQNCiXi48/TDQoxIDAgb2JqDQo8PA0KL1R5cGUgL1BhZ2UNCi9NZWRpYUJveCBbIDAgMCA1OTUuMzA0IDg0MS44OSBdDQovUmVzb3VyY2VzIDw8IC9YT2JqZWN0IDw8IC9JbTQxIDIgMCBSID4+IC9Gb250IDw8IC8= (truncated),"I am writing to report on my recent surveillance findings regarding illegal activities within various crime families. Giulia Bianchi, a key figure in the Bianchi organization, has been observed exchanging encrypted messages with Federico Romano of the Romano syndicate. These communications suggest a potential collaboration between the two families to expand their illicit enterprises. Furthermore, intercepted conversations between Elena Conti from the Conti Crime Family and Francesco Ricci indicate negotiations for a large-scale drug trafficking operation. It is evident that these families are actively seeking to strengthen their positions in the criminal underworld. Additionally, during my investigations, I uncovered disturbing information regarding Luca Moretti, who appears to be operating an illegal Balsamico Vinegar factory that adulterates the finest Tuscan Balsamico vinegar. The shipments are facilitated by Giuseppe Rossi, who acts as a business associate in transporting goods from Peru to Italy. This operation poses significant risks to public health and safety and must be addressed promptly. On a personal note, amidst the complexities of my undercover work, I must confess to finding solace in the indulgence of Italian Grappa spirit and embracing the ""Dolce Vita"" lifestyle. While it may seem trivial in the context of my duties, it serves as a reminder of the finer pleasures amidst the darkness of criminal activities. I will continue to gather intelligence and keep you informed of any significant developments."
dbfs:/FileStore/crime_letters/communication_15_4_2013.pdf,2024-05-06T22:50:58.000+0000,91833,JVBERi0xLjQNCiXi48/TDQoxIDAgb2JqDQo8PA0KL1R5cGUgL1BhZ2UNCi9NZWRpYUJveCBbIDAgMCA1OTUuMzA0IDg0MS44OSBdDQovUmVzb3VyY2VzIDw8IC9YT2JqZWN0IDw8IC9JbTIzIDIgMCBSID4+IC9Gb250IDw8IC8= (truncated),"I am writing to update you on my ongoing undercover investigation into various criminal syndicates. Recent surveillance has revealed interactions between Sofia Rossi of the Rossi Crime Family and Carlo Romano from the Romano organization. These meetings suggest a potential partnership between the two families to expand their criminal enterprises. Additionally, intercepted communications between Giovanni Moretti of the Moretti syndicate and Francesca Marini from the Marini family indicate discussions regarding the smuggling of contraband goods. It is crucial that we take decisive action to disrupt these illegal activities and dismantle the alliances forming between the crime families. I will continue to gather intelligence and provide further updates as necessary."
dbfs:/FileStore/crime_letters/communication_2_3_2013.pdf,2024-05-06T22:50:57.000+0000,89489,JVBERi0xLjQNCiXi48/TDQoxIDAgb2JqDQo8PA0KL1R5cGUgL1BhZ2UNCi9NZWRpYUJveCBbIDAgMCA1OTUuMzA0IDg0MS44OSBdDQovUmVzb3VyY2VzIDw8IC9YT2JqZWN0IDw8IC9JbTUgMiAwIFIgPj4gL0ZvbnQgPDwgL0Y= (truncated),"I am writing to provide updates on my infiltration into the criminal underworld. During my surveillance, I have observed interactions between members of various families. Giuseppe Rossi, head of the Rossi Crime Family, has been seen meeting with Marco Bianchi from the Bianchi syndicate. This rendezvous raises concerns about a potential collaboration between the two families. Additionally, there have been intercepted communications between Francesco Ricci of the Ricci Crime Family and Teresa Marini from the Marini organization, discussing the distribution of illegal goods. Such alliances could significantly impact organized crime dynamics in the region. I will continue to monitor these developments closely and provide further updates as necessary."
dbfs:/FileStore/crime_letters/communi

#### Named Entity Recognition
Named Entity Recognition (NER) is a natural language processing (NLP) technique that identifies and categorizes named entities within text into predefined categories such as names of persons, organizations, locations and dates. Our informant, who has infiltrated the organization, is sending regular letters. You are tasked with building a prototype of automatically processing the reports and extracting the names of the people in each letter, which can then be related to the larger network. 

We will define manually a pipeline that transforms our report text first into an embedding representation and then extracts our entities.

In [0]:
# Step 1: Transforms raw texts to "document" annotation
documentAssembler = DocumentAssembler()\
    .setInputCol("report_text")\
    .setOutputCol("document")

# Step 2: Tokenization
tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

# Step 3: Get the embeddings using glove_100d
embeddings = WordEmbeddingsModel.pretrained("glove_100d").\
                  setInputCols(["document", "token"]).\
                  setOutputCol("embeddings")

# Step 4: Use the ``ner_dl`` model
public_ner = NerDLModel.pretrained("ner_dl", "en") \
          .setInputCols(["document", "token", "embeddings"]) \
          .setOutputCol("ner")

# Step 5: Convert to NER
ner_converter = NerConverter() \
                .setInputCols(["document", "token", "ner"]) \
                  .setOutputCol("entities")

# Define the pipeline
ner_pipeline = Pipeline(stages=[ documentAssembler, 
                                 tokenizer,
                                 embeddings,
                                 public_ner,
                                 ner_converter
                                 ])

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]
ner_dl download started this may take some time.
Approximate size to download 13.6 MB
[OK!]


``Pipelines`` are a Spark concept that we will revisit during our labs on Machine Learning. You will find the API similar to that of ``scikit-learn`` in that it follows the fit/transform structure. Using pipelines, you may specify the steps of a sequence. Most often, you will find yourself using them 

In [0]:
# We fit our model 
ner_pipeline_model = ner_pipeline.fit(mafia_network_communications)

# And transform the data
processed = ner_pipeline_model.transform(mafia_network_communications)

In [0]:
ner_results = processed \
    .select(F.col("ner"), F.col("path"))

Let us now inspect the results.

In [0]:
ner_results.limit(1).display()

ner path List(List(named_entity, 0, 0, O, Map(word -> I, sentence -> 0), List()), List(named_entity, 2, 3, O, Map(word -> am, sentence -> 0), List()), List(named_entity, 5, 11, O, Map(word -> writing, sentence -> 0), List()), List(named_entity, 13, 14, O, Map(word -> to, sentence -> 0), List()), List(named_entity, 16, 21, O, Map(word -> report, sentence -> 0), List()), List(named_entity, 23, 24, O, Map(word -> on, sentence -> 0), List()), List(named_entity, 26, 27, O, Map(word -> my, sentence -> 0), List()), List(named_entity, 29, 34, O, Map(word -> recent, sentence -> 0), List()), List(named_entity, 36, 47, O, Map(word -> surveillance, sentence -> 0), List()), List(named_entity, 49, 56, O, Map(word -> findings, sentence -> 0), List()), List(named_entity, 58, 66, O, Map(word -> regarding, sentence -> 0), List()), List(named_entity, 68, 74, O, Map(word -> illegal, sentence -> 0), List()), List(named_entity, 78, 87, O, Map(word -> activities, sentence -> 0), List()), List(named_entity, 89, 94, O, Map(word -> within, sentence -> 0), List()), List(named_entity, 96, 102, O, Map(word -> various, sentence -> 0), List()), List(named_entity, 104, 108, O, Map(word -> crime, sentence -> 0), List()), List(named_entity, 110, 117, O, Map(word -> families, sentence -> 0), List()), List(named_entity, 118, 118, O, Map(word -> ., sentence -> 0), List()), List(named_entity, 120, 125, B-PER, Map(word -> Giulia, sentence -> 0), List()), List(named_entity, 127, 133, I-PER, Map(word -> Bianchi, sentence -> 0), List()), List(named_entity, 134, 134, O, Map(word -> ,, sentence -> 0), List()), List(named_entity, 136, 136, O, Map(word -> a, sentence -> 0), List()), List(named_entity, 138, 140, O, Map(word -> key, sentence -> 0), List()), List(named_entity, 142, 147, O, Map(word -> figure, sentence -> 0), List()), List(named_entity, 149, 150, O, Map(word -> in, sentence -> 0), List()), List(named_entity, 153, 155, O, Map(word -> the, sentence -> 0), List()), List(named_entity, 157, 163, B-ORG, Map(word -> Bianchi, sentence -> 0), List()), List(named_entity, 165, 176, O, Map(word -> organization, sentence -> 0), List()), List(named_entity, 177, 177, O, Map(word -> ,, sentence -> 0), List()), List(named_entity, 179, 181, O, Map(word -> has, sentence -> 0), List()), List(named_entity, 183, 186, O, Map(word -> been, sentence -> 0), List()), List(named_entity, 188, 195, O, Map(word -> observed, sentence -> 0), List()), List(named_entity, 197, 206, O, Map(word -> exchanging, sentence -> 0), List()), List(named_entity, 208, 216, O, Map(word -> encrypted, sentence -> 0), List()), List(named_entity, 218, 225, O, Map(word -> messages, sentence -> 0), List()), List(named_entity, 228, 231, O, Map(word -> with, sentence -> 0), List()), List(named_entity, 233, 240, B-PER, Map(word -> Federico, sentence -> 0), List()), List(named_entity, 242, 247, I-PER, Map(word -> Romano, sentence -> 0), List()), List(named_entity, 249, 250, O, Map(word -> of, sentence -> 0), List()), List(named_entity, 252, 254, O, Map(word -> the, sentence -> 0), List()), List(named_entity, 256, 261, B-PER, Map(word -> Romano, sentence -> 0), List()), List(named_entity, 263, 271, O, Map(word -> syndicate, sentence -> 0), List()), List(named_entity, 272, 272, O, Map(word -> ., sentence -> 0), List()), List(named_entity, 274, 278, O, Map(word -> These, sentence -> 0), List()), List(named_entity, 280, 293, O, Map(word -> communications, sentence -> 0), List()), List(named_entity, 295, 301, O, Map(word -> suggest, sentence -> 0), List()), List(named_entity, 305, 305, O, Map(word -> a, sentence -> 0), List()), List(named_entity, 307, 315, O, Map(word -> potential, sentence -> 0), List()), List(named_entity, 317, 329, O, Map(word -> collaboration, sentence -> 0), List()), List(named_entity, 331, 337, O, Map(word -> between, sentence -> 0), List()), List(named_entity, 339, 341, O, Map(word -> the, sentence -> 0), List()), List(named_entity, 343, 345, O, Map(word -> two, sentence -> 0), List()), List(named_

To facilitate our analysis, we will use the ``path`` column to extract the date of the report and use the date to assign a report id. We will accomplish this by using a Window function. From there, we explode the array of struct (the result of the NER) and retrieve only the entity and the associated word.

In [0]:
from pyspark.sql.window import Window

In [0]:
# 1. We extract the date
# 2. Get a row ID based on date
# 3. Explode the column
# 4. Extract the results (in a struct)

reports_parsed = ner_results \
    .withColumn("date", F.to_date(F.regexp_extract(F.col("path"), r"(\d{4}-\d+-\d{2})", 1))) \
    .withColumn("report_id", F.row_number().over(Window.orderBy("date"))) \
    .withColumn("ner_exploded", F.explode("ner")) \
    .withColumns({
        "result":  F.col("ner_exploded.result"), 
        "metadata": F.col("ner_exploded.metadata.word") 
    }
    ) \
    .withColumn("row_number", F.row_number().over(Window.orderBy("report_id"))) \
    .select(F.col("result"), F.col("metadata"), F.col("report_id"), F.col("row_number"))

In [0]:
reports_parsed.limit(50).display()

result,metadata,report_id,row_number
O,I,1,1
O,am,1,2
O,writing,1,3
O,to,1,4
O,report,1,5
O,on,1,6
O,my,1,7
O,recent,1,8
O,surveillance,1,9
O,findings,1,10


We know that our agent always spells out the full name of the persons he follows (how convenient!). This allows us to do a clever join to get the people's full name: We join the DataFrame onto itself on the newly created variable ``row_number``, where the left side corresponds to the first name and the right side to the last name. Since we know the ordering we have as the join key (``row_number``, ``row_number - 1``).

In [0]:
sub_network = reports_parsed.alias("df1").withColumnRenamed("metadata", "First Name").join(
    reports_parsed.alias("df2").withColumnRenamed("metadata", "Last Name"),
    (F.col("df1.row_number") == F.col("df2.row_number") - 1) & 
    (F.col("df1.result") == "B-PER") & 
    (F.col("df2.result") == "I-PER"),
    "inner"
)

display(sub_network)

result,First Name,report_id,row_number,result,Last Name,report_id,row_number
B-PER,Giulia,1,19,I-PER,Bianchi,1,20
B-PER,Federico,1,37,I-PER,Romano,1,38
B-PER,Elena,1,65,I-PER,Conti,1,66
B-PER,Francesco,1,73,I-PER,Ricci,1,74
B-PER,Luca,1,113,I-PER,Moretti,1,114
B-PER,Giuseppe,1,139,I-PER,Rossi,1,140
B-PER,Carlo,2,280,I-PER,Romano,2,281
B-PER,Giovanni,2,308,I-PER,Moretti,2,309
B-PER,Francesca,2,315,I-PER,Marini,2,316
B-PER,Giuseppe,3,394,I-PER,Rossi,3,395


We have now extracted a subnet of the overall Mafia network. Our tasks are now two-fold:

1. Which is the individual with the highest influence within the sub-network (based on each relation type)?
2. Which is the individual with the highest influence within the overall network (based on each relation type)?

Both of these question can be answered using network analysis!

In [0]:
from graphframes import * 

nodes = spark.read.csv("dbfs:/FileStore/mafia_nodes.csv", header=True)
edges = spark.read.csv("dbfs:/FileStore/mafia_edges.csv", header=True)


We will create an index column for the relation types.

In [0]:
import pyspark.sql.types as tp

edges = edges \
    .withColumn("relation_type_index", F.dense_rank().over(Window.orderBy("relation_type"))) \
    .withColumn("weight", F.col("weight").cast(tp.IntegerType()))
edges.display()

src,dst,relation_type,weight,relation_type_index
1,2,Asked for Meeting With,1,1
3,4,Asked for Meeting With,2,1
4,1,Asked for Meeting With,1,1
5,6,Asked for Meeting With,1,1
5,7,Asked for Meeting With,1,1
6,5,Asked for Meeting With,2,1
7,5,Asked for Meeting With,1,1
8,7,Asked for Meeting With,1,1
9,12,Asked for Meeting With,1,1
11,12,Asked for Meeting With,2,1


Now we instantiate our complete graph.

In [0]:
mafia_graph = GraphFrame(nodes, edges)

/databricks/spark/python/pyspark/sql/dataframe.py:170: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


In [0]:
# Let us inspect the graph
mafia_graph.vertices.show()
mafia_graph.edges.show()

+---+----------+---------+--------------+
| id|First Name|Last Name|        Family|
+---+----------+---------+--------------+
|  1|  Giuseppe|    Rossi|  Rossi Family|
|  2|     Maria|    Rossi|  Rossi Family|
|  3|   Antonio|    Rossi|  Rossi Family|
|  4|     Sofia|    Rossi|  Rossi Family|
|  5|      Luca|  Bianchi|Bianchi Family|
|  6|    Giulia|  Bianchi|Bianchi Family|
|  7|     Marco|  Bianchi|Bianchi Family|
|  8|   Alessia|  Bianchi|Bianchi Family|
|  9| Francesco|    Ricci|  Ricci Family|
| 10|    Chiara|    Ricci|  Ricci Family|
| 11|   Roberto|    Ricci|  Ricci Family|
| 12|     Laura|    Ricci|  Ricci Family|
| 13|  Giovanni|  Moretti|Moretti Family|
| 14|      Anna|  Moretti|Moretti Family|
| 15|    Matteo|  Moretti|Moretti Family|
| 16|     Elena|  Moretti|Moretti Family|
| 17|     Carlo|   Romano| Romano Family|
| 18|     Lucia|   Romano| Romano Family|
| 19|  Federico|   Romano| Romano Family|
| 20|   Martina|   Romano| Romano Family|
+---+----------+---------+--------

In [0]:
import pyspark.sql.functions as F
mafia_graph.edges.select(F.col("relation_type")).distinct().show()

+--------------------+
|       relation_type|
+--------------------+
|Asked for Meeting...|
|          Threatened|
|          Sent Money|
|              Called|
+--------------------+



We now subset our overall ``nodes`` DataFrame to extract the sub-network only.

In [0]:
sub_network_nodes = mafia_graph.vertices \
    .join(sub_network, on=["First Name", "Last Name"], how="inner") \
    .dropDuplicates(["First Name", "Last Name"])

display(sub_network_nodes)

First Name,Last Name,id,Family,result,report_id,row_number,result,report_id,row_number
Antonio,Rossi,3,Rossi Family,B-PER,7,898,I-PER,7,899
Carlo,Romano,17,Romano Family,B-PER,2,280,I-PER,2,281
Chiara,Ricci,10,Ricci Family,B-PER,7,906,I-PER,7,907
Elena,Conti,26,Conti Family,B-PER,1,65,I-PER,1,66
Federico,Romano,19,Romano Family,B-PER,1,37,I-PER,1,38
Francesca,Marini,24,Marini Family,B-PER,2,315,I-PER,2,316
Francesco,Ricci,9,Ricci Family,B-PER,1,73,I-PER,1,74
Giovanni,Moretti,13,Moretti Family,B-PER,2,308,I-PER,2,309
Giulia,Bianchi,6,Bianchi Family,B-PER,1,19,I-PER,1,20
Giuliana,Conti,28,Conti Family,B-PER,6,818,I-PER,6,819


In [0]:
# mafia_graph_edges_sub_network = sub_network_nodes \
#     .withColumn("edge_id", F.concat(F.col("src"), F.col("dst"), F.col("row_number"))) \
#     .selectExpr("edge_id", 'explode(array(src, dst)) AS node_in_edge') \
#     .dropDuplicates()

# mafia_graph_edges_sub_network.display()

edge_id,node_in_edge
121,1
121,2
341,3
341,4
411,4
411,1
561,5
561,6
571,5
571,7


In [0]:
# Get unique edges from subnetwork
sub_network_edges = list(map(lambda x: x["id"], sub_network_nodes.select("id").collect()))

# Filter network by subnetwork nodes
sub_network_df = mafia_graph.filterEdges(F.col("src").isin(sub_network_edges) | F.col("dst").isin(sub_network_edges)).edges

/databricks/spark/python/pyspark/sql/dataframe.py:149: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")
/databricks/spark/python/pyspark/sql/dataframe.py:170: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


In [0]:
mafia_subgraph = GraphFrame(sub_network_nodes, sub_network_df)

mafia_subgraph.vertices.show()

/databricks/spark/python/pyspark/sql/dataframe.py:170: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


+----------+---------+---+--------------+------+---------+----------+------+---------+----------+
|First Name|Last Name| id|        Family|result|report_id|row_number|result|report_id|row_number|
+----------+---------+---+--------------+------+---------+----------+------+---------+----------+
|   Antonio|    Rossi|  3|  Rossi Family| B-PER|        7|       898| I-PER|        7|       899|
|     Carlo|   Romano| 17| Romano Family| B-PER|        2|       280| I-PER|        2|       281|
|    Chiara|    Ricci| 10|  Ricci Family| B-PER|        7|       906| I-PER|        7|       907|
|     Elena|    Conti| 26|  Conti Family| B-PER|        1|        65| I-PER|        1|        66|
|  Federico|   Romano| 19| Romano Family| B-PER|        1|        37| I-PER|        1|        38|
| Francesca|   Marini| 24| Marini Family| B-PER|        2|       315| I-PER|        2|       316|
| Francesco|    Ricci|  9|  Ricci Family| B-PER|        1|        73| I-PER|        1|        74|
|  Giovanni|  Morett

While ``graphframes`` offers a method to compute ``degree centrality``, its inventory is relatively limited. Since we are operating on different partitions of the graph (i.e. the subgraph induced by the mentioned people in the reports and the entire graph), we can use Spark's capabilities and parallelize the operations. To this end, we will make use of the ``networkx`` library, which offers a wealth of functions to work with graphs.

As we can see, there are four different types of edges:
- Asked for Meeting
- Threatened
- Sent Money
- Called

Those edge types give us the idea that this graph is directed.

Let us now proceed to our actual network analysis. We will compute to network centrality measures here, (in-/out-)degree centrality and betweenness centrality.

- *Degree centrality* measures the importance of a node in a network based on its connections. In the context of in-degree centrality, this metric quantifies how many incoming connections a node has, reflecting its popularity or influence within the network. Conversely, out-degree centrality assesses the number of outgoing connections from a node, indicating its capacity to disseminate information or influence others. 

- *Betweenness centrality*, on the other hand, evaluates the extent to which a node serves as a bridge or intermediary between other nodes in the network. Nodes with high betweenness centrality often lie on many shortest paths between pairs of nodes, suggesting their critical role in maintaining connectivity and facilitating communication within the network.

Both of these, among many others, are implement in ``networkx``. To make us of these functionalities, we use a trick we learned in a previous class: ``pandas`` UDFs, which allow us to pass our graphframe or dataframe into a function and operate on it as normal Python code.

In [0]:
import pandas as pd

output_schema_degree = tp.StructType([
    tp.StructField("relation_type", tp.StringType(), False),
    tp.StructField("node", tp.StringType(), False),
    tp.StructField("in_degree_centrality", tp.FloatType(), False),
    tp.StructField("out_degree_centrality", tp.FloatType(), False),
])

output_schema_betweenness = tp.StructType([
    tp.StructField("relation_type", tp.StringType(), False),
    tp.StructField("node", tp.StringType(), False),
    tp.StructField("betweenness_centrality", tp.FloatType(), False),
])

def nx_degree_centrality(pdf: pd.DataFrame) -> pd.DataFrame:
    # We get the relation_type key
    key = pdf["relation_type"].iloc[0]

    # Here we instantiate a networkx directed graph (DiGraph)
    in_degree_centralities = nx.in_degree_centrality(nx.DiGraph(nx.from_pandas_edgelist(pdf, "src", "dst", edge_attr="weight")))
    out_degree_centralities = nx.out_degree_centrality(nx.DiGraph(nx.from_pandas_edgelist(pdf, "src", "dst", edge_attr="weight")))
    
    # Finally, we return a pd.DataFrame
    return pd.DataFrame(
        {
            "relation_type": [key for _ in range(1, len(in_degree_centralities.values()) + 1)], 
            "node": in_degree_centralities.keys(), 
            "in_degree_centrality": in_degree_centralities.values(),
            "out_degree_centrality": out_degree_centralities.values(),
        }
    )

def nx_betweenness_centrality(pdf: pd.DataFrame) -> pd.DataFrame:
    # We get the relation_type key
    key = pdf["relation_type"].iloc[0]

    # Here we instantiate a networkx directed graph (DiGraph)
    betweenness_centrality = nx.betweenness_centrality(nx.DiGraph(nx.from_pandas_edgelist(pdf, "src", "dst", edge_attr="weight")))
    
    # Finally, we return a pd.DataFrame
    return pd.DataFrame(
        {
            "relation_type": [key for _ in range(1, len(betweenness_centrality.values()) + 1)], 
            "node": betweenness_centrality.keys(), 
            "betweenness_centrality": betweenness_centrality.values()
        }
    )

We compute degree centrality and betweenness centrality for the subgraph

In [0]:
# Summarize results for degree centrality
mafia_subgraph \
    .edges \
    .groupby("relation_type") \
    .applyInPandas(nx_degree_centrality, output_schema_degree) \
    .alias("degree_df") \
    .join(mafia_graph.vertices.alias("node_df"), F.col("degree_df.node") == F.col("node_df.id")) \
    .orderBy("relation_type", "in_degree_centrality", ascending=False) \
    .display()

relation_type,node,in_degree_centrality,out_degree_centrality,id,First Name,Last Name,Family
Threatened,2,0.07692308,0.07692308,2,Maria,Rossi,Rossi Family
Threatened,1,0.07692308,0.07692308,1,Giuseppe,Rossi,Rossi Family
Threatened,4,0.07692308,0.07692308,4,Sofia,Rossi,Rossi Family
Threatened,3,0.07692308,0.07692308,3,Antonio,Rossi,Rossi Family
Threatened,10,0.07692308,0.07692308,10,Chiara,Ricci,Ricci Family
Threatened,9,0.07692308,0.07692308,9,Francesco,Ricci,Ricci Family
Threatened,12,0.07692308,0.07692308,12,Laura,Ricci,Ricci Family
Threatened,11,0.07692308,0.07692308,11,Roberto,Ricci,Ricci Family
Threatened,30,0.07692308,0.07692308,30,Paola,Rossi,Rossi Family
Threatened,29,0.07692308,0.07692308,29,Giorgio,Rossi,Rossi Family


In [0]:
# Summarize results for betweenness centrality
mafia_subgraph \
    .edges \
    .groupby("relation_type") \
    .applyInPandas(nx_betweenness_centrality, output_schema_betweenness) \
    .alias("degree_df") \
    .join(mafia_graph.vertices.alias("node_df"), F.col("degree_df.node") == F.col("node_df.id")) \
    .orderBy("relation_type", "betweenness_centrality", ascending=False) \
    .display()

relation_type,node,betweenness_centrality,id,First Name,Last Name,Family
Threatened,2,0.0,2,Maria,Rossi,Rossi Family
Threatened,1,0.0,1,Giuseppe,Rossi,Rossi Family
Threatened,4,0.0,4,Sofia,Rossi,Rossi Family
Threatened,3,0.0,3,Antonio,Rossi,Rossi Family
Threatened,10,0.0,10,Chiara,Ricci,Ricci Family
Threatened,9,0.0,9,Francesco,Ricci,Ricci Family
Threatened,12,0.0,12,Laura,Ricci,Ricci Family
Threatened,11,0.0,11,Roberto,Ricci,Ricci Family
Threatened,30,0.0,30,Paola,Rossi,Rossi Family
Threatened,29,0.0,29,Giorgio,Rossi,Rossi Family


We repeat this exercise for the entire graph.

In [0]:
# Summarize results for degree centrality
mafia_graph \
    .edges \
    .groupby("relation_type") \
    .applyInPandas(nx_degree_centrality, output_schema_degree) \
    .alias("degree_df") \
    .join(mafia_graph.vertices.alias("node_df"), F.col("degree_df.node") == F.col("node_df.id")) \
    .orderBy("relation_type", "in_degree_centrality", ascending=False) \
    .display()

relation_type,node,in_degree_centrality,out_degree_centrality,id,First Name,Last Name,Family
Threatened,2,0.07692308,0.07692308,2,Maria,Rossi,Rossi Family
Threatened,1,0.07692308,0.07692308,1,Giuseppe,Rossi,Rossi Family
Threatened,4,0.07692308,0.07692308,4,Sofia,Rossi,Rossi Family
Threatened,3,0.07692308,0.07692308,3,Antonio,Rossi,Rossi Family
Threatened,10,0.07692308,0.07692308,10,Chiara,Ricci,Ricci Family
Threatened,9,0.07692308,0.07692308,9,Francesco,Ricci,Ricci Family
Threatened,12,0.07692308,0.07692308,12,Laura,Ricci,Ricci Family
Threatened,11,0.07692308,0.07692308,11,Roberto,Ricci,Ricci Family
Threatened,30,0.07692308,0.07692308,30,Paola,Rossi,Rossi Family
Threatened,29,0.07692308,0.07692308,29,Giorgio,Rossi,Rossi Family


In [0]:
# Summarize results for betweenness centrality
mafia_graph \
    .edges \
    .groupby("relation_type") \
    .applyInPandas(nx_betweenness_centrality, output_schema_betweenness) \
    .alias("degree_df") \
    .join(mafia_graph.vertices.alias("node_df"), F.col("degree_df.node") == F.col("node_df.id")) \
    .orderBy("relation_type", "betweenness_centrality", ascending=False) \
    .display()

relation_type,node,betweenness_centrality,id,First Name,Last Name,Family
Threatened,2,0.0,2,Maria,Rossi,Rossi Family
Threatened,1,0.0,1,Giuseppe,Rossi,Rossi Family
Threatened,4,0.0,4,Sofia,Rossi,Rossi Family
Threatened,3,0.0,3,Antonio,Rossi,Rossi Family
Threatened,10,0.0,10,Chiara,Ricci,Ricci Family
Threatened,9,0.0,9,Francesco,Ricci,Ricci Family
Threatened,12,0.0,12,Laura,Ricci,Ricci Family
Threatened,11,0.0,11,Roberto,Ricci,Ricci Family
Threatened,30,0.0,30,Paola,Rossi,Rossi Family
Threatened,29,0.0,29,Giorgio,Rossi,Rossi Family


#### Bonus
If would like to compute the overall influence, without regard to the relation type, you can use a *fictional* relation type by creating a column with a literal value, such as 1 or a string. Note that using networkx does not leverage the speed of Spark, unless we partition our network in some way. Since we have a network with weights, we can use a groupby to sum the weight, which automatically imposes a uniqueness condition. We still need our fictional groupby to use ``applyInPandas``.

In [0]:
mafia_graph \
    .edges \
    .withColumn("relation_type", F.lit("1")) \
    .groupby(["src", "dst", "relation_type"]) \
    .agg(F.sum("weight").alias("weight")) \
    .groupby("relation_type") \
    .applyInPandas(nx_degree_centrality, output_schema_degree) \
    .alias("degree_df") \
    .join(mafia_graph.vertices.alias("node_df"), F.col("degree_df.node") == F.col("node_df.id")) \
    .orderBy("relation_type", "in_degree_centrality", ascending=False) \
    .display()

relation_type,node,in_degree_centrality,out_degree_centrality,id,First Name,Last Name,Family
1,9,0.29090908,0.29090908,9,Francesco,Ricci,Ricci Family
1,13,0.29090908,0.29090908,13,Giovanni,Moretti,Moretti Family
1,1,0.29090908,0.29090908,1,Giuseppe,Rossi,Rossi Family
1,3,0.29090908,0.29090908,3,Antonio,Rossi,Rossi Family
1,6,0.27272728,0.27272728,6,Giulia,Bianchi,Bianchi Family
1,8,0.27272728,0.27272728,8,Alessia,Bianchi,Bianchi Family
1,11,0.27272728,0.27272728,11,Roberto,Ricci,Ricci Family
1,4,0.27272728,0.27272728,4,Sofia,Rossi,Rossi Family
1,7,0.27272728,0.27272728,7,Marco,Bianchi,Bianchi Family
1,5,0.27272728,0.27272728,5,Luca,Bianchi,Bianchi Family
